In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import os
from PIL import Image
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F 

In [2]:
class SNN(nn.Module):
    def __init__(self, basenet):
        super().__init__()
        self.basenet = basenet
        self.linear1 = nn.Linear(1000,10)
        self.linear2 = nn.Linear(1000,10)
        
        self.cos = nn.CosineSimilarity()
        self.sigmoid = nn.Sigmoid()
    def forward(self, pos, neg):
        res = []
        # Siamese nets; sharing weights
        x = self.basenet(pos)
        res.append(self.linear1(x))
        x = self.basenet(neg)
        res.append(self.linear2(x))
        
        score = self.sigmoid(self.cos(res[0],res[1]))
        
        return score

In [3]:
# Setup: initialize the hyperparameters/variables
# Setup: initialize the hyperparameters/variables
num_epochs = 10           # Number of full passes through the dataset
batch_size = 64       # Number of samples in each minibatch
learning_rate = 0.1  
seed = np.random.seed(0) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

margin = 0.4

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here

transform = transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.ToPILImage('L'),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(30),
        #transforms.RandomRotation(60),
        transforms.Resize([28,28],interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [4]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [5]:
import torchvision.models as models
res = models.resnet18(pretrained=True)
res.avgpool = nn.AdaptiveAvgPool2d(1)
res.classifier = Identity()
model = SNN(res)

In [6]:

# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 6, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

model = model
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

CUDA is supported
Model on CUDA? True


In [7]:
### need to modify
# Setup the training, validation, and testing dataloaders
import quickdraw_dataloader as qd
from quickdraw_dataloader import create_split_loaders
root_dir = "../data_combine/"
train_loader, val_loader, test_loader = create_split_loaders(root_dir,batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

In [10]:
import adabound
#criterion = nn.CrossEntropyLoss().to(computing_device)
criterion = nn.MarginRankingLoss(margin = margin).to(computing_device)

#optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [11]:
def validate(val_loader,model,optimizer,criterion):
    start = time.time()
    sum_loss = 0.0
    list_sum_loss = []
    correct_overall = 0.0
    minbatch_size_ovarall = 0.0
    num = 0
    for mb_count, (val_images, val_labels) in enumerate(val_loader, 0):
        model.eval()
        with torch.no_grad():  
            optimizer.zero_grad()      
            val_images = torch.squeeze(torch.stack([val_images,val_images,val_images], dim=1, out=None))
            val_images, val_labels = val_images.to(computing_device), val_labels.to(computing_device)
            val_labels = val_labels.type(torch.cuda.FloatTensor)
            val_labels = val_labels.long()
            outputs = model(val_images)
            
            loss = criterion(outputs,val_labels)
            sum_loss += loss
            
            output_np = outputs.cpu().detach().numpy()
            label_np = val_labels.cpu().detach().numpy()
            correct_prediction, minbatch_size = correctness(label_np, output_np)
            correct_overall += correct_prediction
            minbatch_size_ovarall += minbatch_size
    accuracy_vali = correct_overall/minbatch_size_ovarall
    print('validatation accuracy',accuracy_vali) 
    
    print("validation time = ", time.time()-start)    
    return 1.0*sum_loss/mb_count, accuracy_vali

In [12]:
def correctness(out1, out2):
    return np.sum((out1-out2)>margin), out1.shape[0]
def train_model(model, criterion,optimizer, scheduler, num_epochs=10,resume=False, direct=""):
    since = time.time()
    total_loss = []
    if resume == False:
        avg_minibatch_loss = []
        total_vali_loss = []
        avg_training_accuracy = []
        avg_validation_accuracy = []
        epc_save = 0 # MAYBE
    else: 
        print('Resume model: %s' % direct)
        check_point = torch.load(direct)
        model.load_state_dict(check_point['state_dict'])
        optimizer.load_state_dict(check_point['optimizer'])
        avg_minibatch_loss = list(np.load('avg_minibatch_loss.npy'))
        total_vali_loss = list(np.load('total_vali_loss.npy'))
        avg_training_accuracy = list(np.load('avg_training_accuracy.npy'))
        avg_validation_accuracy = list(np.load('avg_validation_accuracy.npy'))
        epc_save = check_point['epoch']
        
    tolerence = 3
    i = 0 
    for epoch in range(epc_save,num_epochs):
        N = 100
        M = 100000
        N_minibatch_loss = 0.0    
        best_loss = 100
        early_stop = 0
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        correct_overall = 0.0; 
        minbatch_size_overall = 0.0; 
        # Each epoch has a training and validation phase
        for phase in ['train']:
            scheduler.step()
            # Iterate over data.
            minibatch_time = time.time()
            target = torch.ones(batch_size).to(computing_device)
            for minibatch_count, (pos_0,pos_1, neg) in enumerate(train_loader, 0):
                pos_0 = torch.squeeze(torch.stack([pos_0,pos_0,pos_0], dim=1, out=None)).to(computing_device)
                pos_1 = torch.squeeze(torch.stack([pos_1,pos_1,pos_1], dim=1, out=None)).to(computing_device)
                neg = torch.squeeze(torch.stack([neg,neg,neg], dim=1, out=None)).to(computing_device)
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs_pos = model(pos_0,pos_1)
                    outputs_neg = model(pos_0,neg)
                    #loss = (outputs_neg - outputs_pos + 0.2).mean()
                    loss = criterion(outputs_pos,outputs_neg,target)
                    N_minibatch_loss += loss
                    # backward + optimize only if in training phase
                    loss.backward()
                    optimizer.step()
                    outputs_pos_np = outputs_pos.cpu().detach().numpy()
                    outputs_neg_np = outputs_neg.cpu().detach().numpy()
                    #print(label_np[1:5])

                    correct_prediction, minbatch_size = correctness(outputs_pos_np, outputs_neg_np)
                    correct_overall += correct_prediction
                    minbatch_size_overall += minbatch_size
                # statistics
                # training stats
                if minibatch_count % N == 0 and minibatch_count!=0:    

                    # Print the loss averaged over the last N mini-batches    
                    N_minibatch_loss /= N
                    print('Epoch %d, average minibatch %d loss: %.3f' %
                        (epoch + 1, minibatch_count, N_minibatch_loss))
                    accuracy_train = correct_overall/minbatch_size_overall
                    print('accuracy',accuracy_train)
                    print("pos")
                    print(outputs_pos.mean())
                    print("neg")
                    print(outputs_neg.mean())
                    
                    # Add the averaged loss over N minibatches and reset the counter
                    avg_minibatch_loss.append(N_minibatch_loss)
                    avg_minibatch_loss_1 = np.array(avg_minibatch_loss)
                    #np.save('avg_minibatch_loss', avg_minibatch_loss_1)
                    #avg_training_accuracy.append(accuracy_train)
                    #avg_training_accuracy_1 = np.array(avg_training_accuracy)
                    #np.save('avg_training_accuracy', avg_training_accuracy)
                    
                    N_minibatch_loss = 0.0
                    correct_overall = 0.0
                    minbatch_size_overall = 0.0
                    #print('accuracy, precision, recall', accuracy, precision, recall)
                    save_checkpoint({'epoch': epoch + 1,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict(),
                                },
                                filename='./checkpoint/'+'%d_model_epoch%d.pth' % (epoch , minibatch_count))
                    print(N, "minibatch_time" , time.time()-minibatch_time)
                    minibatch_time = time.time()
                #Validation
#                 if minibatch_count % M == 0 and minibatch_count!=0:
                   
#                     v_loss, accuracy_vali = validate(val_loader,model,optimizer,criterion)
#                     print(v_loss.item())
                    
#                     total_vali_loss.append(v_loss.item())
#                     avg_validation_accuracy.append(accuracy_vali.item())
                    
#                     total_vali_loss_1 = np.array(total_vali_loss)
#                     np.save('total_vali_loss', total_vali_loss_1)      
#                     avg_validation_accuracy_1 = np.array(avg_validation_accuracy)
#                     np.save('avg_validation_accuracy', avg_validation_accuracy_1)      
                
#                     if total_vali_loss[i] > best_loss and i != 0:
#                         early_stop += 1
#                         if early_stop == tolerence:
#                             print('early stop here')
#                             break
#                     else:
#                         best_loss = total_vali_loss[i] 
#                         early_stop = 0
#                     i = i + 1
            print("Finished", epoch + 1, "epochs of training")
    print("Training complete after", epoch, "epochs")
    
    print("total_vali_loss")
    print(total_vali_loss)
    print("avg_minibatch_loss")
    print(avg_minibatch_loss)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s '.format(
        time_elapsed // 60, time_elapsed % 60))


In [13]:
def save_checkpoint(state, is_best=0, filename='models/checkpoint.pth.tar'):
    torch.save(state, filename)         

In [ ]:
model_trained = train_model(model, criterion,optimizer, exp_lr_scheduler, num_epochs=20,resume=False,direct='./checkpoint/4_model_epoch2600.pth')


Epoch 0/19
----------
Epoch 1, average minibatch 100 loss: 0.184
accuracy 0.10550742574257425
pos
tensor(0.7265, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4808, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 82.94691634178162
Epoch 1, average minibatch 200 loss: 0.170
accuracy 0.0378125
pos
tensor(0.7293, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4993, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 77.85517454147339
Epoch 1, average minibatch 300 loss: 0.168
accuracy 0.02921875
pos
tensor(0.7295, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.5180, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 76.11269736289978
Epoch 1, average minibatch 400 loss: 0.169
accuracy 0.02609375
pos
tensor(0.7303, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4847, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 76.68541502952576
Epoch 1, average minibatch 500 loss: 0.167
accuracy 0.016875
pos
tensor(0.7304, de

100 minibatch_time 78.08874773979187
Epoch 1, average minibatch 3800 loss: 0.161
accuracy 0.00328125
pos
tensor(0.7296, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.5202, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 79.51268315315247
Epoch 1, average minibatch 3900 loss: 0.164
accuracy 0.0034375
pos
tensor(0.7306, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4846, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 75.12973213195801
Epoch 1, average minibatch 4000 loss: 0.161
accuracy 0.003125
pos
tensor(0.7301, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.5014, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 77.73069453239441
Epoch 1, average minibatch 4100 loss: 0.161
accuracy 0.00359375
pos
tensor(0.7307, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4917, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 78.74428534507751
Epoch 1, average minibatch 4200 loss: 0.162
accuracy 0.00375
pos
tensor(0.

Epoch 1, average minibatch 7600 loss: 0.162
accuracy 0.005
pos
tensor(0.7310, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4862, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 74.75314378738403
Epoch 1, average minibatch 7700 loss: 0.164
accuracy 0.006875
pos
tensor(0.7306, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.4790, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 74.2221326828003
Epoch 1, average minibatch 7800 loss: 0.162
accuracy 0.0071875
pos
tensor(0.7309, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.5010, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 74.84983777999878
Epoch 1, average minibatch 7900 loss: 0.162
accuracy 0.0040625
pos
tensor(0.7308, device='cuda:0', grad_fn=<MeanBackward1>)
neg
tensor(0.5022, device='cuda:0', grad_fn=<MeanBackward1>)
100 minibatch_time 71.95773530006409
Epoch 1, average minibatch 8000 loss: 0.163
accuracy 0.00359375
pos
tensor(0.7308, device='cuda:0', grad_fn=<MeanBackw